In [2]:
import os
import pandas as pd
import numpy as np
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation


import time
import tqdm
import jpype
import ast

In [3]:
# 전처리, 명사추출, 키워드추출, 불용어후처리, 닉네임추출까지 마친 데이터

import pandas as pd

folder_path = '/Users/jaesolshin/Documents/GitHub/youtube_dashboard'
file_path = os.path.join(folder_path, 'KPOP_comments_merged_preprocessed_after_cleaning_nickname.csv')
comments_df = pd.read_csv(file_path)
sample_df = comments_df.copy()

# 특정 그룹에 대해 분석 수행
sample_df = sample_df[sample_df['Group']=='NMIXX']

# csv 파일로 저장되면서 문자열로 변형된 word_list 컬럼을 다시 리스트로 변환
sample_df['word_list'] = sample_df['word_list'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
sample_df['word_list'] = sample_df['word_list'].apply(lambda x: " ".join(x))
print(len(sample_df))

# word_list가 빈 문자열인 행 제거
sample_df = sample_df[sample_df['word_list'] != '']  # 빈 문자열인 행 제거
print(len(sample_df))

#  members 컬럼을 다시 리스트로 변환
sample_df['members'] = sample_df['members'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

23511
20160


In [20]:
# TF-IDF 벡터화 (전체 데이터에 대해 벡터화 수행)
tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # 최대 1000개의 단어만 사용
tfidf_matrix = tfidf_vectorizer.fit_transform(sample_df['word_list'])

# LDA 모델 학습 (전체 데이터에 대해 학습)
n_topics = n_top_words = 20  # 원하는 토픽 수 설정
lda = LatentDirichletAllocation(n_components=n_topics, 
                                max_iter=5,
                                topic_word_prior=0.1,
                                doc_topic_prior=1.0,
                                learning_method='online',
                                n_jobs=-1,
                                random_state=42)

lda.fit(tfidf_matrix)

# 각 댓글에 대해 토픽 분포 추출 (댓글별로 토픽 분포 계산)
topic_distributions = lda.transform(tfidf_matrix)

# 'topic_distribution' 열에 토픽 분포 저장
sample_df['topic_distribution'] = list(topic_distributions)

def print_top_words(model, feature_names, n_top_words=n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"\n===== 토픽 {topic_idx + 1} =====")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))

tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()
print_top_words(lda, tfidf_feature_names, n_top_words)


===== 토픽 1 =====
별로 노래 무대 시간 엔믹스 음방 나라 리듬 디스 후렴 뮤비 홍콩 콘서트 거슬러 노랜 응원 컨셉 굳이 드뎌 나중

===== 토픽 2 =====
음악 노래 엔믹스 실망 신인 권모술수 하루 가수 반응 신경 호불호 대중 올해 표절 매번 평가 오해원 가창력 사랑해 미래

===== 토픽 3 =====
엔믹스 노래 라이브 이해 응원 친구 수준 가즈아 혼란 구리 실력 사랑해 리스닝 체인지 밴드 주식 컴백 완성 도전 구림

===== 토픽 4 =====
정신 노래 목소리 지우 엔믹스 도입 해원 뮤비 영어 사랑해 해외 하이라이트 멤버 릴리 수고 표정 살짝 설윤 실력 국내

===== 토픽 5 =====
중독 엔믹스 자꾸 얼마나 별별별 케이팝 이상 노래 거지 주년 버전 오오 무조건 재생 뭐임 최근 후렴구 안티 구나 컨셉

===== 토픽 6 =====
엔믹스 파트 재능 노래 핑크 릴리 퀄리티 컴백 왤케 찰떡 상큼 웅장 정체 구성 체인지업 그녀 소속사 제왑 기대 띵곡

===== 토픽 7 =====
지니 노래 한국어 박진영 엔믹스 음원 파이팅 작곡 대형 엔깅이들 자막 고생 기획사 준비 썸네일 여러 안무가 멜론 세대 감성

===== 토픽 8 =====
장르 엔믹스 선공 노래 머리 윤아 도대체 컴백 뮤비 제왑 사랑해 언제 음색 배진솔 대쉬 본부 스타 노력 테마 킬링

===== 토픽 9 =====
분위기 타이틀곡 타이틀 수록곡 아이돌 마음 약간 댄스 최고 코디 엔믹스 노래 장규진 원더걸스 오해원 커버 걱정 이곡 홍보 게뭐

===== 토픽 10 =====
컨셉 트와이스 노래 소름 해원 엔믹스 문제 갑자기 기분 개성 포기 가요 작곡가 뮤비 장면 관심 멤버 이미지 꽃길 외국인

===== 토픽 11 =====
역대 노래 여름 다이스 실험 엔믹스 벌써 티저 화이팅 몇번 금발 아아 옛날 무엇 겨울 설윤 성적 대기업 갈수록 론데

===== 토픽 12 =====
노래 화이팅 엔믹스 에스파 여자 설윤 언니 명곡 헤메 제목 극락 외국 사운드 활동 스키 센터 

In [21]:

# Title별로 토픽 분포의 평균 계산
title_topic_distribution = sample_df.groupby('Title')['topic_distribution'].apply(lambda x: sum(x) / len(x))

# 정의된 토픽이름(결과 보고 이름붙임)
topic_names = {
    1: "무대 연출과 스타일",
    2: "음악 평가와 반응",
    3: "실력과 라이브 퍼포먼스",
    4: "멤버와 팬 감정",
    5: "중독성 있는 케이팝",
    6: "퀄리티와 개인 파트",
    7: "음원 성적과 대중 반응",
    8: "장르와 뮤직비디오 분석",
    9: "타이틀곡과 댄스 퍼포먼스",
    10: "컨셉과 비주얼 매력",
    11: "성과와 활동 평가",
    12: "명곡과 인기 요소",
    13: "진심과 감정 표현",
    14: "안무와 외모 평가",
    15: "데뷔와 인기 상승",
    16: "비주얼과 스타일링",
    17: "보컬 능력과 조회수",
    18: "걸그룹과 퍼포먼스 성공",
    19: "매력과 컴백 활동",
    20: "세계관과 충격 요소"
}


# Title별 상위 세 개의 토픽 분포 출력
for title, topic_dist in title_topic_distribution.items():
    print(f"\n===== {title} 그룹의 상위 3개 토픽 분포 =====")
    
    # 토픽 분포를 (토픽 번호, 확률) 형식으로 정렬
    sorted_topic_dist = sorted(enumerate(topic_dist, 1), key=lambda x: x[1], reverse=True)
    
    # 상위 3개 토픽만 출력
    for i, prob in sorted_topic_dist[:3]:
        print(f"{topic_names[i]}: {prob:.4f}")


# 첫 번째 샘플의 topic_distribution에서 최대값의 인덱스를 찾기
def get_topic_name(value):
    # 토픽 분포 중 최대값의 인덱스를 찾고, 그에 해당하는 토픽 이름 반환
    idx = np.argmax(value) + 1  # 1을 더해서 topic_names에 맞는 인덱스로 변환
    return topic_names[idx]

# 댓글의 topic_distribution에서 상위 3개의 토픽 이름을 얻기
def get_topic_names(value):
    # 토픽 분포를 내림차순으로 정렬한 후 상위 3개의 인덱스를 찾고, 그에 해당하는 토픽 이름을 반환
    top_indices = np.argsort(value)[-3:][::-1]  # 상위 3개 인덱스 (내림차순으로 정렬)
    top_topics = [topic_names[idx + 1] for idx in top_indices]  # 1을 더해서 topic_names에 맞는 인덱스로 변환
    return top_topics


sample_df['topic_distribution'] = sample_df['topic_distribution'].apply(get_topic_name)


===== DASH 그룹의 상위 3개 토픽 분포 =====
장르와 뮤직비디오 분석: 0.0506
진심과 감정 표현: 0.0506
퀄리티와 개인 파트: 0.0506

===== DICE 그룹의 상위 3개 토픽 분포 =====
보컬 능력과 조회수: 0.0508
음원 성적과 대중 반응: 0.0505
안무와 외모 평가: 0.0503

===== Funky Glitter Christmas 그룹의 상위 3개 토픽 분포 =====
음원 성적과 대중 반응: 0.0516
음악 평가와 반응: 0.0513
퀄리티와 개인 파트: 0.0512

===== Love Me Like This 그룹의 상위 3개 토픽 분포 =====
무대 연출과 스타일: 0.0510
중독성 있는 케이팝: 0.0506
타이틀곡과 댄스 퍼포먼스: 0.0505

===== O.O 그룹의 상위 3개 토픽 분포 =====
데뷔와 인기 상승: 0.0520
진심과 감정 표현: 0.0507
명곡과 인기 요소: 0.0504

===== Party O’Clock 그룹의 상위 3개 토픽 분포 =====
성과와 활동 평가: 0.0512
명곡과 인기 요소: 0.0508
컨셉과 비주얼 매력: 0.0506

===== Roller Coaster 그룹의 상위 3개 토픽 분포 =====
성과와 활동 평가: 0.0515
세계관과 충격 요소: 0.0508
퀄리티와 개인 파트: 0.0506

===== Soñar (Breaker) 그룹의 상위 3개 토픽 분포 =====
장르와 뮤직비디오 분석: 0.0508
음악 평가와 반응: 0.0507
퀄리티와 개인 파트: 0.0507

===== Young, Dumb, Stupid 그룹의 상위 3개 토픽 분포 =====
멤버와 팬 감정: 0.0516
안무와 외모 평가: 0.0516
타이틀곡과 댄스 퍼포먼스: 0.0512

===== 별별별 (See that) 그룹의 상위 3개 토픽 분포 =====
걸그룹과 퍼포먼스 성공: 0.0508
중독성 있는 케이팝: 0.0505
타이틀곡과 댄스 퍼포먼스

In [22]:
# 출력 옵션 설정: 모든 열을 출력
pd.set_option('display.max_columns', None)  # 모든 열 출력
pd.set_option('display.max_rows', None)     # 모든 행 출력

sample_df[['comment','topic_distribution']].head(200)

,comment,topic_distribution
0,엔믹스 노래 중독성 있고 좋은데 왜 안뜨는지 진심 의문..,중독성 있는 케이팝
1,대쉬랑 비슷한것 같기도하고 이젠 좀 다른 대중성있는 노래로갔으면하는 아쉬움이있네요 ...,안무와 외모 평가
2,4본부장아 피씨로 엘범 클릭하면 별별별 뮤비로 연결 안되고 수정별인가 그 장면 있는...,안무와 외모 평가
3,이야 노래 종나 멋있네,진심과 감정 표현
4,외모췍을 필두로 더더더더 빵빵뜨길...리더로서 해원이 얼마나 노력했을까...멤버들 ...,데뷔와 인기 상승
5,노래는 좋은데 ㅋㅋ 가사는 개짜치는 별별별ㅋㅋㅋㅋ,중독성 있는 케이팝
6,해원이 외모체크 이후로 찐팬됨 멤버 모두 너무 착하고 예쁨 실력도 탑이네,안무와 외모 평가
7,와 진짜 설윤이랑 지우랑 비주얼 폭발하는 노래 하나 타이틀로 내자 정말 엔믹 요즘 ...,성과와 활동 평가
8,[timecode] 이어폰 까고 들으니까 무슨 연극같음,명곡과 인기 요소
9,ㅈ구림,실력과 라이브 퍼포먼스


In [23]:
import gensim
from gensim import corpora

# 문서 토큰화 (예시)
documents = sample_df['word_list'].apply(lambda x: x.split())  # 'word_list'는 단어들이 공백으로 구분된 문자열로 가정

# 단어 사전(dictionary) 생성
dictionary = corpora.Dictionary(documents)

# 문서를 Bag of Words 형태로 변환 (단어: 빈도수)
corpus = [dictionary.doc2bow(doc) for doc in documents]

# LDA 모델 학습
NUM_TOPICS = 20  # 20개의 토픽 설정
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, passes=15)

# 각 토픽의 상위 단어 출력
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)


(0, '0.184*"중독" + 0.125*"비주얼" + 0.114*"가사" + 0.044*"무대"')
(1, '0.362*"엔믹스" + 0.094*"실력" + 0.044*"대중성" + 0.036*"세대"')
(2, '0.078*"이해" + 0.051*"능력" + 0.043*"일본" + 0.038*"문제"')
(3, '0.151*"컴백" + 0.086*"엔믹스" + 0.044*"파이팅" + 0.042*"테마"')
(4, '0.153*"릴리" + 0.132*"걸그룹" + 0.086*"보컬" + 0.035*"오해원"')
(5, '0.061*"정신" + 0.044*"소녀" + 0.035*"대체" + 0.033*"뉴진스"')
(6, '0.076*"대중" + 0.055*"에스파" + 0.021*"세계관" + 0.019*"영어"')
(7, '0.577*"노래" + 0.088*"뮤비" + 0.073*"엔믹스" + 0.022*"멤버"')
(8, '0.136*"안무" + 0.055*"충격" + 0.042*"스타" + 0.032*"비쥬"')
(9, '0.197*"컨셉" + 0.082*"음악" + 0.058*"진심" + 0.053*"역대"')
(10, '0.087*"장난" + 0.076*"얼마나" + 0.058*"도대체" + 0.049*"거지"')
(11, '0.089*"제대로" + 0.057*"약간" + 0.052*"도입" + 0.029*"하루"')
(12, '0.108*"멤버" + 0.086*"설윤" + 0.058*"신곡" + 0.055*"매력"')
(13, '0.140*"제왑" + 0.134*"별로" + 0.047*"한국인" + 0.028*"올해"')
(14, '0.148*"노래" + 0.084*"분위기" + 0.079*"장르" + 0.046*"아이돌"')
(15, '0.304*"데뷔" + 0.078*"트와이스" + 0.047*"기대" + 0.036*"걸그룹"')
(16, '0.158*"화이팅" + 0.117*"엔믹스" + 0.040*"댄스" + 0.028*"머리"')
(1